In [2]:
##Create a spark session
from pyspark.sql import SparkSession
# Create a SparkSession
spark = SparkSession.builder.appName("Read CSV").getOrCreate()


In [13]:
users = spark.read.csv("raw/books_data/users.csv",sep=";",header=True)

[('User-ID', 'string'), ('Location', 'string'), ('Age', 'string')]


In [21]:
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as F
from pyspark.sql.functions import mean
users = users.withColumn("Age", users["Age"].cast(IntegerType()))
users = users.withColumn("User-ID", users["User-ID"].cast(IntegerType()))

[('User-ID', 'int'), ('Location', 'string'), ('Age', 'int')]

In [38]:

# Split the column into a list of strings
df = users.withColumn("Location", F.split(users.Location, ","))
# Filter the rows that do not have the correct format
df = df.filter(F.size(df.Location) == 3)
df = df.withColumn("city", F.col("Location").getItem(0))
df = df.withColumn("state", F.col("Location").getItem(1))
df = df.withColumn("country", F.col("Location").getItem(2))
df = df.drop("Location")
# Calculate the mean of the column
mean_value = df.agg(mean(df.Age)).collect()[0][0]

# Fill the NA values in the column with the mean
df = df.fillna(mean_value, subset=["Age"])
# Print the DataFrame
df.show()

+-------+---+--------------+----------------+---------------+
|User-ID|Age|          city|           state|        country|
+-------+---+--------------+----------------+---------------+
|      1| 34|           nyc|        new york|            usa|
|      2| 18|      stockton|      california|            usa|
|      3| 34|        moscow| yukon territory|         russia|
|      4| 17|         porto|        v.n.gaia|       portugal|
|      5| 34|   farnborough|           hants| united kingdom|
|      6| 61|  santa monica|      california|            usa|
|      7| 34|    washington|              dc|            usa|
|      8| 34|       timmins|         ontario|         canada|
|      9| 34|    germantown|       tennessee|            usa|
|     10| 26|      albacete|       wisconsin|          spain|
|     11| 14|     melbourne|        victoria|      australia|
|     12| 34|    fort bragg|      california|            usa|
|     13| 26|     barcelona|       barcelona|          spain|
|     14